# Exploring EBay Car Sales Data

### Context
In this project, we'll be working with a dataset of used cars from [eBay Kleinanzeigen](https://www.ebay-kleinanzeigen.de/), a classifieds section of the German eBay website.

The data set was originally scraped and uploaded to [Kaggle](https://www.kaggle.com/orgesleka/used-cars-database/data), along with the following data dictionary:

- `dateCrawled` - When this ad was first crawled. All field values are taken from this date.
- `name` - Name of the car.
- `seller` - Whether the seller is private or a dealer.
- `offerType` - The type of listing
- `price` - The price on the ad to sell the car.
- `abtest` - Whether the listing is included in an A/B test.
- `vehicleType` - The vehicle Type.
- `yearOfRegistration` - The year in which the car was first registered.
- `gearbox` - The transmission type.
- `powerPS` - The power of the car in PS.
- `model` - The car model name.
- `odometer` - How many kilometers the car has driven.
- `monthOfRegistration` - The month in which the car was first registered.
- `fuelType` - What type of fuel the car uses.
- `brand` - The brand of the car.
- `notRepairedDamage` - If the car has a damage which is not yet repaired.
- `dateCreated` - The date on which the eBay listing was created.
- `nrOfPictures` - The number of pictures in the ad.
- `postalCode` - The postal code for the location of the vehicle.
- `lastSeenOnline` - When the crawler saw this ad last online.

### Aim
The aim of this project is to clean, explore, analyse and understand the data for these used car listings. 

We'll be using a variety of basic pandas methods to do so.


## Import the Data 

Let's start by importing the libraries we need and reading the data set into pandas.

In [1]:
import numpy as np
import pandas as pd

autos = pd.read_csv('autos.csv', encoding='Latin-1')
print(autos.info())
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


From the above we can make some initial observations.

- The data set contains 50,000 rows and 20 columns
- There are over 20,000 null values across 5 columns. No one column has more than 20% null values
- Data types are either `ints` (25%) or `strings` (75%). Data relating to dates, price and mileage are in string format
- Column `name` data does not appear to follow a particular formatting convention. Names are quite long and can contain underscores, capitals, lowercase, forward slashes
- Column names are in camelCase, but are not consistent, e.g. `abtest`

Let's take a closer look at the column names.

In [2]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

## Clean Column Names

For the sake of consistency and readability we decide to clean these column names. We will change the format to `snake_case` and edit some special cases.

To help us out with this, we will use a [regular expression](https://en.wikipedia.org/wiki/Regular_expression) function we found on [Stack Overflow](https://stackoverflow.com/questions/1175208/elegant-python-function-to-convert-camelcase-to-snake-case), have tested out and found to be suitable for our purpose! The function converts `camelCase` to `snake_case`.

In [3]:
# Make a copy
columns_copy = autos.columns.copy()

# Import regular expression module
import re

# Function to convert from camelcase to snakecase
def camel_to_snake(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

# Iterate over column_copy and update new list
col_names = []

for col in columns_copy:
    col = camel_to_snake(col)
    col_names.append(col)

# Update the special cases
col_names[5] = 'ab_test'
col_names[7] = 'registration_year'
col_names[12] = 'registration_month'
col_names[-5] = 'unrepaired_damage'
col_names[-4] = 'ad_created'
col_names[-3] = 'num_pics'

# Assign the new list back to autos.columns
autos.columns = col_names

# Check
print(autos.columns)
autos.head()

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_pics', 'postal_code',
       'last_seen'],
      dtype='object')


,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_pics,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


## Initial Exploration and Cleaning

Let's get some descriptive statistics for our data using `autos.describe()`. We note the following:

- Columns to drop  
`num_pics` column appears to have one value only, `0`. This column can be dropped as it doesn't provide any useful information for analysis.

- Numeric data as text  
This applies to columns such as `price`, `odometer`, and for data relating to dates, e.g. `date_crawled` and `ad_created`.

- Columns for further investigation  
`seller`, `offer_type`, `ab_test` and `gearbox` have very low variance with only 2 unique values per column.


In [4]:
# Get stats for both text and numerical data
print(autos.describe(include='all'))


               date_crawled         name  seller offer_type  price ab_test  \
count                 50000        50000   50000      50000  50000   50000   
unique                48213        38754       2          2   2357       2   
top     2016-03-30 17:37:35  Ford_Fiesta  privat    Angebot     $0    test   
freq                      3           78   49999      49999   1421   25756   
mean                    NaN          NaN     NaN        NaN    NaN     NaN   
std                     NaN          NaN     NaN        NaN    NaN     NaN   
min                     NaN          NaN     NaN        NaN    NaN     NaN   
25%                     NaN          NaN     NaN        NaN    NaN     NaN   
50%                     NaN          NaN     NaN        NaN    NaN     NaN   
75%                     NaN          NaN     NaN        NaN    NaN     NaN   
max                     NaN          NaN     NaN        NaN    NaN     NaN   

       vehicle_type  registration_year  gearbox      power_ps  

Let's take a closer look at `seller`, `offer_type`, `ab_test` and `gearbox`.

Upon inspection, the binary nature of both `ab_test` and `gearbox`, along with the distributions of their values, makes intuitive sense and does not raise any red flags with us. 

However `seller` and `offer_type` are more concerning, as they are almost entirely comprised of a single value.

As such we decide to keep `ab_test` and `gearbox`, but drop `seller` and `offer_type`. We will also drop `num_pics` which, as noted above, also contains only one single value.

In [5]:
# Check count of unique values
binary = ['seller', 'offer_type', 'ab_test', 'gearbox']

for col in binary:
    print('------------')
    print(col)
    print(autos[col].value_counts())    

# Drop cols with no variability in values
autos.drop(['seller', 'offer_type', 'num_pics'], axis=1, inplace=True)

# Check
autos.head()

------------
seller
privat        49999
gewerblich        1
Name: seller, dtype: int64
------------
offer_type
Angebot    49999
Gesuch         1
Name: offer_type, dtype: int64
------------
ab_test
test       25756
control    24244
Name: ab_test, dtype: int64
------------
gearbox
manuell      36993
automatik    10327
Name: gearbox, dtype: int64


,date_crawled,name,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50


`price` and `odometer` are two columns of interest to us. 

To make them usable we'll need to clean them and convert them from text to numeric.

In [6]:
# Remove '$' and ',' and convert to float
autos['price'] = autos['price'].str.replace('$', '').str.replace(',', '').astype(float)

# Remove ',' and 'km' and convert to float
autos['odometer'] = autos['odometer'].str.replace(',', '').str.replace('km', '').astype(float)

# Rename column to `odometer_km`
autos.rename({'odometer' : 'odometer_km'}, axis=1, inplace=True)

# Check
print(autos.columns)
print(autos['price'].describe())
print(autos['odometer_km'].describe())

Index(['date_crawled', 'name', 'price', 'ab_test', 'vehicle_type',
       'registration_year', 'gearbox', 'power_ps', 'model', 'odometer_km',
       'registration_month', 'fuel_type', 'brand', 'unrepaired_damage',
       'ad_created', 'postal_code', 'last_seen'],
      dtype='object')
count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64
count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64


## Exploring the Odometer and Price Columns

In the `odometer_km` column the values are all rounded off at multiples of 5000 or 10000 km, suggesting they are pre-populated values the owners must select from.

There is a strong correlation between the amount of kilometers on the odometer and the number of cars for sale, which makes intuitive sense - the older or more used a car, the more likely that someone might want to sell or replace it.

In [7]:
autos['odometer_km'].value_counts()

150000.0    32424
125000.0     5170
100000.0     2169
90000.0      1757
80000.0      1436
70000.0      1230
60000.0      1164
50000.0      1027
5000.0        967
40000.0       819
30000.0       789
20000.0       784
10000.0       264
Name: odometer_km, dtype: int64

In the `price` column, the descriptive stats we saw above show us that price points range from \\$0 to $100,000,000. Let's take a closer look at these values.

We can see below that almost 3% of all cars are priced at $0.

At the other end of the spectrum there is a total of 0.028% of cars priced at large and suspicious looking amounts, e.g. \\$1,234,566, \\$123,456,789, $11,111,111 and so on.

In [8]:
# Percentage of prices
autos['price'].value_counts(normalize=True).sort_index()*100


0.0           2.842
1.0           0.312
2.0           0.006
3.0           0.002
5.0           0.004
8.0           0.002
9.0           0.002
10.0          0.014
11.0          0.004
12.0          0.006
13.0          0.004
14.0          0.002
15.0          0.004
17.0          0.006
18.0          0.002
20.0          0.008
25.0          0.010
29.0          0.002
30.0          0.014
35.0          0.002
40.0          0.012
45.0          0.008
47.0          0.002
49.0          0.008
50.0          0.098
55.0          0.004
59.0          0.002
60.0          0.018
65.0          0.010
66.0          0.002
              ...  
151990.0      0.002
155000.0      0.002
163500.0      0.002
163991.0      0.002
169000.0      0.002
169999.0      0.002
175000.0      0.002
180000.0      0.002
190000.0      0.002
194000.0      0.002
197000.0      0.002
198000.0      0.002
220000.0      0.002
250000.0      0.002
259000.0      0.002
265000.0      0.002
295000.0      0.002
299000.0      0.002
345000.0      0.002


Let's take a closer look at the $0 cars. 

There does not seem to be a discernible pattern in the data to indicate why a car might be listed for $0.

In [9]:
zero_price = autos.loc[autos['price'] == 0]

for col in zero_price:
    print('----------')
    print(col)
    print(zero_price[col].value_counts(dropna=False, normalize=True)*100)


----------
date_crawled
2016-03-27 22:55:05    0.140746
2016-03-20 13:42:08    0.140746
2016-03-19 10:50:25    0.140746
2016-03-09 09:50:57    0.140746
2016-03-28 17:54:44    0.140746
2016-03-30 20:57:06    0.140746
2016-03-18 20:44:43    0.140746
2016-03-17 22:37:24    0.070373
2016-03-07 11:53:29    0.070373
2016-03-29 20:51:37    0.070373
2016-04-01 20:52:12    0.070373
2016-03-07 07:57:39    0.070373
2016-03-05 18:41:12    0.070373
2016-03-31 02:57:00    0.070373
2016-03-28 19:58:06    0.070373
2016-03-07 21:45:42    0.070373
2016-03-29 20:52:42    0.070373
2016-03-29 15:47:02    0.070373
2016-04-03 13:52:33    0.070373
2016-03-31 12:50:55    0.070373
2016-03-25 12:53:38    0.070373
2016-04-04 03:03:16    0.070373
2016-03-26 23:55:53    0.070373
2016-03-15 11:50:40    0.070373
2016-03-23 22:53:54    0.070373
2016-03-24 21:52:20    0.070373
2016-04-02 00:52:35    0.070373
2016-04-04 15:43:06    0.070373
2016-03-23 23:43:35    0.070373
2016-03-11 18:42:52    0.070373
                

Perhaps these \\$0 rows are legitimate entries? 

We speculate that owners possibly just wanted cars removed, e.g. the cars were written off, and were happy to have them taken off their hands for \\$0. This could be a possibility, as 17.5% of listings admit to unrepaired damage, and a further 50% of these $0 car owners have left the unrepaired damage field empty. 

However, 3%, or 1421 cars, seems like a large number of free cars to be giving away.

Perhaps $0 was an initial or default value set by some owners, and they let the subsequent bids dictate the price the car would sell for?

This idea is not not supported however when we visit the [American](https://www.ebay.com/) and [Australian](https://www.ebay.com.au/) versions of eBay and attempt an advanced search for cars with $0 pricing. We are unable to find any listings with this criterion.

![](ebay_US.png)

Source: [eBay US](https://www.ebay.com/)


![](ebay_Aus1.png)

Source: [eBay Australia](https://www.ebay.com.au/)



We therefore decide to remove these \\$0 rows for now.

Additionally, we will remove rows where price is greater than \\$350,000, due to the large and suspicious looking values returned above this price point.

In [10]:
# Keep rows between $1 and $350K inclusive
autos = autos[autos['price'].between(1,350000)]

# Check
autos['price'].describe()


count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

## Exploring the Date Columns

`date_crawled`, `ad_created` and `last_seen` are all dates represented as strings in the format 'YYYY-MM-DD HH:MM:SS'.

In [11]:
autos[['date_crawled', 'ad_created', 'last_seen']].head()

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


In order to make the dates more readable and to enable our analysis, we decide to split the time data from the date data in each data point. We will then perform our analysis on the date data.

`date_crawled` refers to the dates that the data was retrieved from the site, which as we can see occurred between 5th March and 7th April 2016. 

Posts have a fairly uniform frequency distributed, except for the last 2 days where the frequency tapers off. We can assume this is related to some kind of lag between real time posting and data collection.

In [12]:
# Get distribution of 'date_crawled' dates as percentages
crawled_date = autos['date_crawled'].str.split(expand=True)[0]
crawled_date.value_counts(normalize=True).sort_index()*100

2016-03-05    2.532688
2016-03-06    1.404304
2016-03-07    3.601359
2016-03-08    3.329558
2016-03-09    3.308967
2016-03-10    3.218367
2016-03-11    3.257490
2016-03-12    3.691959
2016-03-13    1.566972
2016-03-14    3.654896
2016-03-15    3.428395
2016-03-16    2.960980
2016-03-17    3.162772
2016-03-18    1.291053
2016-03-19    3.477813
2016-03-20    3.788737
2016-03-21    3.737259
2016-03-22    3.298672
2016-03-23    3.222485
2016-03-24    2.934212
2016-03-25    3.160712
2016-03-26    3.220426
2016-03-27    3.109235
2016-03-28    3.486050
2016-03-29    3.409863
2016-03-30    3.368681
2016-03-31    3.183363
2016-04-01    3.368681
2016-04-02    3.547823
2016-04-03    3.860805
2016-04-04    3.648718
2016-04-05    1.309585
2016-04-06    0.317101
2016-04-07    0.140019
Name: 0, dtype: float64

From the `ad_created` dates we can see that many were created quite recently relative to when the data was crawled, within 1-2 months. 

However there are a number of older ads still posted, the furthest back being approximately 9 months old at time of data collection.

In [13]:
# Get distribution of 'ad_created' dates as percentages
created_date = autos['ad_created'].str.split(expand=True)[0]
created_date.value_counts(normalize=True).sort_index()*100

2015-06-11    0.002059
2015-08-10    0.002059
2015-09-09    0.002059
2015-11-10    0.002059
2015-12-05    0.002059
2015-12-30    0.002059
2016-01-03    0.002059
2016-01-07    0.002059
2016-01-10    0.004118
2016-01-13    0.002059
2016-01-14    0.002059
2016-01-16    0.002059
2016-01-22    0.002059
2016-01-27    0.006177
2016-01-29    0.002059
2016-02-01    0.002059
2016-02-02    0.004118
2016-02-05    0.004118
2016-02-07    0.002059
2016-02-08    0.002059
2016-02-09    0.002059
2016-02-11    0.002059
2016-02-12    0.004118
2016-02-14    0.004118
2016-02-16    0.002059
2016-02-17    0.002059
2016-02-18    0.004118
2016-02-19    0.006177
2016-02-20    0.004118
2016-02-21    0.006177
                ...   
2016-03-09    3.315145
2016-03-10    3.189540
2016-03-11    3.290435
2016-03-12    3.675486
2016-03-13    1.700813
2016-03-14    3.518995
2016-03-15    3.401627
2016-03-16    3.012458
2016-03-17    3.127767
2016-03-18    1.359003
2016-03-19    3.368681
2016-03-20    3.794914
2016-03-21 

`last_seen` tells us when the crawler last saw this ad up online. 

Over time, as cars are sold, ads would be taken down. It makes sense that further back, e.g. at the start of the date range below, the frequency distribution is much less than it is at the end of the captured period, where there would still be a higher number of newer ads for cars yet unsold.

In [14]:
# Get distribution of 'last_seen' dates as percentages
lastseen_date = autos['last_seen'].str.split(expand=True)[0]
lastseen_date.value_counts(normalize=True).sort_index()*100

2016-03-05     0.107073
2016-03-06     0.432410
2016-03-07     0.539483
2016-03-08     0.741275
2016-03-09     0.959539
2016-03-10     1.066612
2016-03-11     1.237517
2016-03-12     2.378256
2016-03-13     0.889529
2016-03-14     1.260167
2016-03-15     1.587563
2016-03-16     1.645218
2016-03-17     2.808607
2016-03-18     0.735097
2016-03-19     1.583445
2016-03-20     2.065273
2016-03-21     2.063214
2016-03-22     2.137342
2016-03-23     1.853186
2016-03-24     1.976732
2016-03-25     1.921137
2016-03-26     1.680222
2016-03-27     1.564913
2016-03-28     2.085864
2016-03-29     2.234119
2016-03-30     2.477093
2016-03-31     2.378256
2016-04-01     2.279419
2016-04-02     2.491506
2016-04-03     2.520334
2016-04-04     2.448265
2016-04-05    12.476063
2016-04-06    22.180583
2016-04-07    13.194688
Name: 0, dtype: float64

The details for `registration_year` give us pause. We can see a min value of `1000` and a max value of `9999`.

Let's investigate further.

In [15]:
autos['registration_year'].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

## Dealing with Incorrect Registration Year Data

[Mass production](https://en.wikipedia.org/wiki/Car#History) of cars did not begin until 1901, however we can see 6 posts with registration date pre-1900.

Additionally there are almost 2000 cars dated after 2016, the year this data was collected. 

In [16]:
autos['registration_year'].value_counts().sort_index()

1000       1
1001       1
1111       1
1800       2
1910       5
1927       1
1929       1
1931       1
1934       2
1937       4
1938       1
1939       1
1941       2
1943       1
1948       1
1950       3
1951       2
1952       1
1953       1
1954       2
1955       2
1956       4
1957       2
1958       4
1959       6
1960      23
1961       6
1962       4
1963       8
1964      12
        ... 
2000    3156
2001    2636
2002    2486
2003    2699
2004    2703
2005    2936
2006    2670
2007    2277
2008    2215
2009    2085
2010    1589
2011    1623
2012    1310
2013     803
2014     663
2015     392
2016    1220
2017    1392
2018     470
2019       2
2800       1
4100       1
4500       1
4800       1
5000       4
5911       1
6200       1
8888       1
9000       1
9999       3
Name: registration_year, Length: 95, dtype: int64

We can safely remove rows falling into these 2 scenarios - pre-1900 and post-2016.

In [17]:
autos = autos[autos['registration_year'].between(1900,2016)]

## Exploring Price by Brand

Let's explore the price points of the top 10 most-sold car brands.

First let's get our top 10 selling cars brands.

In [18]:
top_10 = autos['brand'].value_counts(normalize=True, dropna=False).head(10)
top_10

volkswagen       0.211264
bmw              0.110045
opel             0.107581
mercedes_benz    0.096463
audi             0.086566
ford             0.069900
renault          0.047150
peugeot          0.029841
fiat             0.025642
seat             0.018273
Name: brand, dtype: float64

Now lets aggregate these brands on their average price.

In [19]:
# Initialise a dictionary to store brands and their mean price
brand_dict = {}

# Loop over brands and find their mean price
for car in top_10.index:
    car_brand_df = autos[autos['brand'] == car]
    mean_price = int(car_brand_df['price'].mean())
    brand_dict[car] = mean_price    
    
brand_dict

{'volkswagen': 5402,
 'bmw': 8332,
 'opel': 2975,
 'mercedes_benz': 8628,
 'audi': 9336,
 'ford': 3749,
 'renault': 2474,
 'peugeot': 3094,
 'fiat': 2813,
 'seat': 4397}

Let's put these top 10 most sold brands in order of most expensive to least, according to the brand mean price. 

We can see that the most expensive cars are the high-end models we know such as `Audi`, `Mercedes-Benz` and `BMW`.

The mid-range price points belong to `Volkswagen`, `Seat`, `Ford` and `Peugeot`.

At the lower end of the scale we find `Opel`, `Fiat` and `Renault`. 

In [20]:
# Convert dictionary to Dataframe and sort values descending
brand_df = pd.DataFrame(pd.Series(brand_dict), columns=['mean_price'])
brand_df = brand_df.sort_values('mean_price', ascending=False)
brand_df

,mean_price
audi,9336
mercedes_benz,8628
bmw,8332
volkswagen,5402
seat,4397
ford,3749
peugeot,3094
opel,2975
fiat,2813
renault,2474


## Exploring Odometers

Using the same method as above for mean price, let's now calculate the mean kilometers travelled for the top 10 most sold brands.


In [21]:
# Initialise a dictionary to store brands and their mean kilometers
km_dict = {}

# Loop over brands and find their mean kilometers travelled
for car in top_10.index:
    car_brands_df = autos[autos['brand'] == car]
    mean_km = int(car_brands_df['odometer_km'].mean())
    km_dict[car] = mean_km
    
km_dict

{'volkswagen': 128707,
 'bmw': 132572,
 'opel': 129310,
 'mercedes_benz': 130788,
 'audi': 129157,
 'ford': 124266,
 'renault': 128071,
 'peugeot': 127153,
 'fiat': 117121,
 'seat': 121131}

Now let's examine the mean kilometers travelled and the mean prices for each brand.

In [22]:
# Convert dictionary to Series and add to dataframe
brand_df['mean_km'] = pd.Series(km_dict)
brand_df

,mean_price,mean_km
audi,9336,129157
mercedes_benz,8628,130788
bmw,8332,132572
volkswagen,5402,128707
seat,4397,121131
ford,3749,124266
peugeot,3094,127153
opel,2975,129310
fiat,2813,117121
renault,2474,128071


If looking at 'mileage' in isolation, we might assume cars with high mileage would be at the cheaper end of the scale.

However, interestingly, some of the brands for sale with the highest number of kilometers on the odometers were also those with the highest mean price points.

`BMW` - Most well-travelled  
`Mercedes-Benz` - 2nd most well-travelled  
`Audi` - 4th most well-travelled


In [23]:
brand_df.sort_values('mean_km', ascending=False)

,mean_price,mean_km
bmw,8332,132572
mercedes_benz,8628,130788
opel,2975,129310
audi,9336,129157
volkswagen,5402,128707
renault,2474,128071
peugeot,3094,127153
ford,3749,124266
seat,4397,121131
fiat,2813,117121


# Conclusion

In this project, we used a variety of pandas methods to explore, clean, analyse and understand a data set on car listings.